# 멋진 작사가 만들기


먼저 사용할 라이브러리들을 호출해줍니다.

데이터를 준비하고 내용을 확인해봅니다.

파일을 읽기 모드로 열어준 다음 라인으로 끊어서 리스트 형태로 읽습니다.

In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


앞선 예제에서 다루어 본 것처럼 화자가 표기된 문장이나 공백인 문장이 있을 수 있습니다.

그렇기에 길이가 0인 문장을 건너뛰어서 공백인 문장을 제외하고,

문장의 끝부분이 :인 문장을 건너뛰어서 화자가 표기된 문장을 제외해 줍니다.

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개를 확인해 봅니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


## 토큰화

문장을 일정한 기준으로 나누어주는 과정을 토큰화라고 합니다.

여기에서는 띄어쓰기를 기준으로 나누어 줄 것입니다.

토큰화하기 쉽게 전처리하는 함수를 만듭니다.

함수를 살펴보면, 대소문자를 다르게 인식하는 것을 방지하기 위해 모든 문자를 소문자로 바꿉니다.

문장을 정확하게 분리하기 위해서 문장 부호 양쪽에 공백을 추가합니다.

공백이 여러개인 부분은 하나로 바꿔줍니다.

a-zA-Z?.!,¿가 아닌 다른 문자들은 공백으로 바꿉니다.

이후 다시 양쪽의 공백을 지워준 다음

문장의 시작 부분에 <start>, 끝 부분에 <end>를 붙여줍니다.



In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6

    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


raw_corpus의 문장들 중 공백인 문장과 화자를 지칭하는 문장을 제외하고,

문장들을 앞서 만든 전처리 함수를 통해 전처리 해준 다음, corpus 리스트에 추가해줍니다.

전처리가 끝난 10개의 문장을 출력해서 살펴보겠습니다.

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

이제 단어를 기억할 수 있는 tokenize 함수를 만들어 줍니다.

12000개의 단어를 기억할 수 있게 num_words=12000으로 지정해주고,

12000개의 단어에 포함되지 못한 단어는 <unk>로 바꿉니다.

앞서 전처리를 했기 때문에 filter는 필요 없기에 ' '으로 합니다.

전처리된 corpus를 사용해서 단어장을 만들어줍니다.

tokenizer을 이용해서 corpus를 Tensor로 변환해줍니다.

문장의 끝에 padding이 오게끔 padding='post'로 지정한 다음,

maxlen=15로 지정합니다.(작사할 때, 긴 문장이 잘 쓰이지 않기 때문입니다.)

In [5]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    print(tensor.shape)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7efbf43973d0>
(175749, 15)


텐서 데이터의 예시를 출력해서 확인합니다.

텐서 데이터는 모두 정수로 이루어짐을 알 수 있습니다.

In [6]:
print(tensor[:5, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]
 [   2   11  354   25   42    3    0    0    0    0]
 [   2    6 3604    4    6 2265    3    0    0    0]]


tokenizer에 만들어진 단어 사전을 확인합니다.

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


앞서 확인한 텐서 데이터의 모든 행이 2로 시작했었는데 <start>가 2라서 그렇다는 것을 알 수 있겠네요.

텐서를 소스와 타겟으로 분리합니다.

마지막 토큰은 <pad>일 가능성이 높기에 잘라내고 소스 문장을 만들고,

<start>를 잘라내고 타켓 문장을 만듭니다.

In [8]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


데이터셋 객체를 생성합니다.

텐서플로우를 활용합니다.

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   # + 1은 <pad>

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 학습시키기

1개의 Embdding Layer, 2개의 LSTM layer, 1개의 Dense Layer로 구성된 인공지능입니다.

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # word 벡터의 차원 수, 단어가 추상적으로 표현되는 크기
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

한 배치를 불러와서 데이터를 모델에 넣어봅니다.

In [11]:
for src_sample, tgt_sample in dataset.take(1): break

lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.07528035e-04,  7.43569544e-05, -4.62151293e-05, ...,
         -9.32724797e-05,  3.76116397e-04, -1.74390107e-06],
        [-2.09052756e-04,  3.08914568e-05, -1.03920924e-04, ...,
         -1.77570299e-04,  4.06885025e-04,  3.06748589e-05],
        [-1.36263450e-04,  2.87887582e-04, -2.07461606e-04, ...,
         -4.19581833e-04,  2.99590232e-04,  2.66255316e-04],
        ...,
        [ 6.52049319e-04, -2.42056296e-04,  7.18525378e-04, ...,
         -1.37552066e-04,  1.30059107e-05,  7.16036826e-04],
        [ 9.55698430e-04,  2.53730104e-05,  9.39396268e-04, ...,
         -3.31179508e-05,  2.83836824e-04,  5.74084930e-04],
        [ 1.02949387e-03, -3.12641896e-05,  1.29483861e-03, ...,
          2.97021506e-05,  4.78492962e-04,  2.75904225e-04]],

       [[-2.07528035e-04,  7.43569544e-05, -4.62151293e-05, ...,
         -9.32724797e-05,  3.76116397e-04, -1.74390107e-06],
        [-4.87710022e-05,  1.21912228e-04,  2

모델의 최종 출력 텐서 shape를 확인할 수 있고 앞서 지정해준 값(BATCH SIZE, max_len, VOCAB_SIZE)들이 제대로 들어감을 알 수 있습니다.

이후 summary를 호출해서 모델을 살펴봅니다.

In [12]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


train_test_split을 통해 train data와 test data를 분리합니다.

test_size=0.2로 지정했기에 20%가 test data입니다.

In [13]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2)

In [14]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


모델을 학습시킵니다!

In [15]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 222s 320ms/step - loss: 3.6121
Epoch 2/10
686/686 [==============================] - 116s 169ms/step - loss: 3.1301
Epoch 3/10
686/686 [==============================] - 116s 169ms/step - loss: 2.9402
Epoch 4/10
686/686 [==============================] - 116s 169ms/step - loss: 2.7950
Epoch 5/10
686/686 [==============================] - 117s 170ms/step - loss: 2.6700
Epoch 6/10
686/686 [==============================] - 117s 170ms/step - loss: 2.5547
Epoch 7/10
686/686 [==============================] - 117s 170ms/step - loss: 2.4474
Epoch 8/10
686/686 [==============================] - 116s 169ms/step - loss: 2.3469
Epoch 9/10
686/686 [==============================] - 116s 169ms/step - loss: 2.2516
Epoch 10/10
686/686 [==============================] - 116s 169ms/step - loss: 2.1605


## 평가하기

모델에 시작 문장을 입력하면 시작 문장을 바탕으로 작문을 진행하게 하는 함수를 만듭니다.

In [16]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=15):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

## 회고



자연어처리와 관련된 프로젝트는 처음이라 신기했다.

뭔가 새로운 것들을 배우고 작사 모델이 문장을 만들어 내는 것을 보니 흥미로웠다.

그러나 BUT 쉽지 않았다.

프로젝트를 하면 할수록 갈아 넣어야겠구나 하는 생각이 든다...

계속 조금씩 어려워지는 것 같기는 하지만 힘내보겠다...!

